In [1]:
import soccerdata as sd
import pandas as pd
from tqdm import tqdm
import ScraperFC as sfc
import traceback
import datetime

import warnings

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import pyspark.pandas as ps

[12/26/23 12:12:48] INFO     No custom team name replacements found. You can configure these in       ]8;id=113628;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=283408;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py#84\84]8;;\
                             C:\Users\ksbha\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=349693;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=327900;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py#162\162]8;;\
                             C:\Users\ksbha\soccerdata\config\league_dict.json.                                    

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
from datetime import date

today = date.today()
type(today)


datetime.date

In [3]:
## Create a connection to Snowflake
from sqlalchemy import create_engine
import snowflake.connector

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (14.0.2), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


[12/26/23 12:14:09] WARNING  Failed to import ArrowResult. No Apache Arrow result set format can be    ]8;id=175807;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=497280;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#94\94]8;;\
                             used. ImportError: DLL load failed while importing arrow_iterator: The                
                             specified procedure could not be found.                                               

In [4]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [5]:
pd.set_option('display.max_columns', None)

In [5]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Gegenstats") \
    .getOrCreate()

In [6]:
scraper = sfc.Understat()
def scrape_league_data(years):
    output_df = pd.DataFrame(columns=['№', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA',
       'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'season','league'])
    for league_year in league_years:
        try:
            ooper = scraper.scrape_league_table(year=league_year[1], league=league_year[0])
            season_val = league_year[1]
            ooper['season'] = int(str(int(str(season_val)[2:])-1)+str(season_val)[2:])
            if league_year[0] == 'EPL':
                ooper['league'] = 'ENG-Premier League'
            output_df = pd.concat([output_df, ooper])
            
        except Exception as e:
            print(f"An error occurred for the year {league_years}: {e}")
            traceback.print_exc()
    scraper.close()
    return output_df

league_years = [('EPL', 2022), ('EPL', 2024)]  # Replace with your list of years
out = scrape_league_data(league_years)

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:361: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  columns = pd.read_html(table)[0].columns
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:366: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]
C:\Users\ksbha\AppData\Local\Temp\ipykernel_4572\1491577790.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, ooper])


In [7]:
understat_table = out[['league','Team','season', 'W', 'D', 'L','G','GA', 'xG', 'xGA','NPxG','NPxGA','PPDA',
                       'OPPDA','PTS','xPTS']]

In [8]:
understat_table.loc[understat_table['Team'] == 'Manchester United', 'Team'] = 'Manchester Utd'
understat_table.loc[understat_table['Team'] == 'Newcastle United', 'Team'] = 'Newcastle Utd'
understat_table.loc[understat_table['Team'] == 'Tottenham Hotspur', 'Team'] = 'Tottenham'
understat_table.loc[understat_table['Team'] == 'Wolverhampton Wanderers', 'Team'] = 'Wolves'
understat_table.loc[understat_table['Team'] == 'Nottingham Forest', 'Team'] = "Nott'ham Forest"
understat_table.loc[understat_table['Team'] == 'Leicester', 'Team'] = 'Leicester City'
understat_table.loc[understat_table['Team'] == 'Leeds', 'Team'] = 'Leeds United'
understat_table.loc[understat_table['Team'] == 'Sheffield United', 'Team'] = "Sheffield Utd"
understat_table.loc[understat_table['Team'] == 'Luton', 'Team'] = 'Luton Town'
understat_table.loc[understat_table['Team'] == 'Norwich', 'Team'] = 'Norwich City'

In [9]:
understat_table.sort_values(by=['league','season','Team'], ascending=True, inplace=True)
understat_table.reset_index(drop=True, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_4572\614021855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.sort_values(by=['league','season','Team'], ascending=True, inplace=True)


In [10]:
understat_table.rename(columns={'Team':'TEAM_NAME', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS', 'league':'COMPETITION',
                                'L': 'TEAM_LOSSES', 'xPTS':'TEAM_XPTS', 'PTS':'TEAM_PTS', 'season':'SEASON',
                                'NPxGA':'NPXG_AGAINST', 'xGA':'XG_AGAINST'}, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_4572\2521636844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.rename(columns={'Team':'TEAM_NAME', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS', 'league':'COMPETITION',


In [11]:
understat_table.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [29]:
fbref = sd.FBref(leagues="ENG-Premier League", seasons=["2122","2324"])
fbref_team_standard = fbref.read_team_season_stats(stat_type='standard')
fbref_team_standard_oppo = fbref.read_team_season_stats(stat_type='standard', opponent_stats=True)
fbref_team_shooting = fbref.read_team_season_stats(stat_type='shooting')
fbref_team_passing = fbref.read_team_season_stats(stat_type='passing')
fbref_team_defense = fbref.read_team_season_stats(stat_type='defense')
fbref_team_possession = fbref.read_team_season_stats(stat_type='possession')
fbref_team_misc = fbref.read_team_season_stats(stat_type='misc')
fbref_schedule = fbref.read_schedule()
# fbref_lineups = fbref.read_lineup()

[12/26/23 12:22:19] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\FBref               ]8;id=916171;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=239469;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [30]:
fbref_team_standard = fbref_team_standard.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_standard_oppo = fbref_team_standard_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_shooting = fbref_team_shooting.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_passing = fbref_team_passing.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_defense = fbref_team_defense.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_possession = fbref_team_possession.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_misc = fbref_team_misc.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_schedule = fbref_schedule.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})
# fbref_lineups = fbref_lineups.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})

In [43]:
# Convert the date column to datetime format
fbref_schedule['date'] = pd.to_datetime(fbref_schedule['date'])
# fbref_schedule = fbref_schedule[fbref_schedule['date'] < datetime.datetime(2023, 12, 20)]

# fbref_schedule[fbref_schedule['date'] <= datetime.datetime.now().strftime('%Y-%m-%d')]

In [33]:
def make_season_integer(df):
    indexes = df.index.names
    df = df.reset_index()
    df['SEASON'] = df['SEASON'].astype(int)
    df = df.set_index(indexes)
    return df

fbref_team_standard = make_season_integer(fbref_team_standard)
fbref_team_standard_oppo = make_season_integer(fbref_team_standard_oppo)
fbref_team_shooting = make_season_integer(fbref_team_shooting)
fbref_team_passing = make_season_integer(fbref_team_passing)
fbref_team_defense = make_season_integer(fbref_team_defense)
fbref_team_possession = make_season_integer(fbref_team_possession)
fbref_team_misc = make_season_integer(fbref_team_misc)
fbref_schedule = make_season_integer(fbref_schedule)
# fbref_lineups = make_season_integer(fbref_lineups)

In [34]:
team_matches_played = fbref_team_standard['Playing Time'].MP
team_goals_scored = understat_table.G
team_npxG_90 = fbref_team_standard['Per 90 Minutes'].npxG
team_goals_conceded = understat_table.GA
team_xG_against_90 = fbref_team_standard_oppo['Per 90 Minutes'].xG
team_shots = fbref_team_shooting['Standard'].Sh
team_shotsOT = fbref_team_shooting['Standard'].SoT
team_pass_completed = fbref_team_passing["Total"].Cmp
team_pass_attempted = fbref_team_passing["Total"].Att
team_tackles = fbref_team_defense.Tackles.Tkl
team_tackles_won = fbref_team_defense.Tackles.TklW
team_takeons_attempted = fbref_team_possession['Take-Ons'].Att
team_takeons_completed = fbref_team_possession['Take-Ons'].Succ
team_crossesintoPA = fbref_team_passing['CrsPA']
team_fouls_against = fbref_team_misc['Performance']['Fld']
team_fouls_made = fbref_team_misc['Performance']['Fls']
team_interceptions = fbref_team_defense['Int']
team_blocks_shots = fbref_team_defense['Blocks'].Sh
team_blocks_pass = fbref_team_defense['Blocks'].Pass
team_clearances = fbref_team_defense.Clr
team_aerials_won = fbref_team_misc['Aerial Duels'].Won
team_aerials_lost = fbref_team_misc['Aerial Duels'].Lost

In [35]:
team_names = fbref_team_standard.reset_index()[['TEAM_NAME','url']]
team_names['TEAM_FBREF_ID'] = team_names['url'].apply(lambda x: x.split('/')[3])      ## Extracting team_id from URL
team_names.drop(columns=['url'], inplace=True)
team_names.columns = team_names.columns.droplevel(1)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_4572\2026419047.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  team_names.drop(columns=['url'], inplace=True)


In [36]:
team_names['TEAM_FBREF_ID'] = team_names['TEAM_FBREF_ID'].astype('string')

In [37]:
team_names.drop_duplicates(inplace=True)

In [20]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [21]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

[12/21/23 17:49:33] INFO     Snowflake Connector for Python Version: 3.3.1, Python Version:       ]8;id=474776;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=523253;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#334\334]8;;\
                             3.11.5, Platform: Windows-10-10.0.22000-SP0                                           

                    INFO     This connection is in OCSP Fail Open Mode. TLS Certificates would   ]8;id=918351;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=700987;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1103\1103]8;;\
                             be checked for validity and revocation status. Any other                              
                             Certificate Revocation related exceptions or OCSP Responder                           
                             failures would be disregarded in favor of connectivity.                               

                    INFO     Setting use_openssl_only mode to False                              ]8;id=304843;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=627664;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1121\1121]8;;\

In [22]:
cur = conn.cursor()

In [23]:
cur.execute('SELECT * FROM TEAMS')
team_rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
curr_team_names = pd.DataFrame(team_rows, columns=column_names)

[12/21/23 17:49:38] INFO     query: [SELECT * FROM TEAMS]                                             ]8;id=987675;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=50070;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=649205;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=2751;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 25                                     ]8;id=404096;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=598948;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [24]:
## Below (if not blank) are the teams you need to create LOGOS for
team_names_not_in_curr = team_names[~team_names['TEAM_NAME'].isin(curr_team_names['TEAM_NAME'])]
team_names_not_in_curr

,TEAM_NAME,TEAM_FBREF_ID


In [25]:
team_logos = \
[["943e8050", "https://i.imgur.com/6IDi9XJ.png"], ["e297cd13", "https://i.imgur.com/vHHqKJX.png"],
 ["1df6b87e", "https://i.imgur.com/eeBetFN.png"], ["2abfe087", "https://i.imgur.com/vlOBKzl.png"],
 ["1c781004", "https://i.imgur.com/WLIU9hF.png"]]

In [26]:
team_names = team_names.merge(pd.DataFrame(team_logos, columns=['TEAM_FBREF_ID', 'TEAM_LOGO']), 
                              how='right', on='TEAM_FBREF_ID')

In [27]:
team_names.rename(columns={'TEAM_LOGO':'TEAM_LOGO_URL'}, inplace=True)

In [28]:
team_names

,TEAM_NAME,TEAM_FBREF_ID,TEAM_LOGO_URL
0,Burnley,943e8050,https://i.imgur.com/6IDi9XJ.png
1,Luton Town,e297cd13,https://i.imgur.com/vHHqKJX.png
2,Sheffield Utd,1df6b87e,https://i.imgur.com/eeBetFN.png
3,Watford,2abfe087,https://i.imgur.com/vlOBKzl.png
4,Norwich City,1c781004,https://i.imgur.com/WLIU9hF.png


In [29]:
def upsert_to_snowflake(table_name, dataframe, primary_keys):
    if isinstance(primary_keys, str):
        primary_keys = [primary_keys]

    temp_table_name = f"{table_name}_TEMP"
    success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)

    on_condition = ' AND '.join([f"{table_name}.{pk} = {temp_table_name}.{pk}" for pk in primary_keys])

    update_columns = [col for col in dataframe.columns if col not in primary_keys]
    update_sql = ', '.join([f"{table_name}.{col} = {temp_table_name}.{col}" for col in update_columns])

    merge_sql = f"""
    MERGE INTO {table_name} USING {temp_table_name}
    ON {on_condition}
    """

    if update_columns:
        merge_sql += f"""
        WHEN MATCHED THEN
            UPDATE SET
                {update_sql}
        """

    merge_sql += f"""
    WHEN NOT MATCHED THEN
        INSERT ({', '.join(dataframe.columns)})
        VALUES ({', '.join([f"{temp_table_name}.{col}" for col in dataframe.columns])})
    """

    cur.execute(merge_sql)
    cur.execute(f"DROP TABLE IF EXISTS {temp_table_name}")

    print(f"Upsert completed. {nrows} rows processed.")

In [33]:
upsert_to_snowflake("TEAMS", team_names, ['TEAM_FBREF_ID'])

[12/21/23 17:50:11] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=137620;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=448344;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=300345;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=533613;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=330479;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=346518;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=240999;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=988826;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[12/21/23 17:50:12] INFO     query execution done                                                     ]8;id=294637;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=939594;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[12/21/23 17:50:15] INFO     query execution done                                                     ]8;id=745041;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=294679;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAMS_TEMP]                                 ]8;id=711078;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=994449;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=575164;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=885599;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=93533;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=240348;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 25 rows processed.


In [75]:
cur.execute('SELECT * FROM TEAMS')
team_rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
team_names = pd.DataFrame(team_rows, columns=column_names)

[12/21/23 17:53:15] INFO     query: [SELECT * FROM TEAMS]                                             ]8;id=556928;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=222472;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[12/21/23 17:53:16] INFO     query execution done                                                     ]8;id=599225;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=546889;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 25                                     ]8;id=187914;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=236764;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [76]:
team_names.drop(columns=['TEAM_LOGO_URL'], inplace=True)

In [31]:
team_standard_stats = understat_table.copy()

In [32]:
team_standard_stats['MATCHES_PLAYED'] = team_matches_played

In [34]:
team_standard_stats.reset_index(inplace=True)

In [35]:
team_standard_stats = team_standard_stats.merge(team_names, on='TEAM_NAME', how='left')

In [36]:
team_standard_stats = team_standard_stats[['TEAM_FBREF_ID','SEASON','COMPETITION','MATCHES_PLAYED','TEAM_WINS', 
                                           'TEAM_DRAWS', 'TEAM_LOSSES', 'TEAM_PTS', 'TEAM_XPTS']]

In [ ]:
upsert_to_snowflake("TEAM_STANDARD_STATS", team_standard_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

In [37]:
team_attacking_stats = understat_table.copy()

In [38]:
team_attacking_stats = team_attacking_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [39]:
team_attacking_stats = team_attacking_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON','NPxG','xG']]

In [40]:
team_attacking_stats.rename(columns={'NPxG': 'NPXG', 'xG':'XG'}, inplace=True)

In [41]:
team_attacking_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [42]:
team_attacking_stats['GOALS_SCORED'] = team_goals_scored
team_attacking_stats['SHOTS'] = team_shots
team_attacking_stats['SHOTS_ON_TARGET'] = team_shotsOT
team_attacking_stats['PASS_COMPLETED'] = team_pass_completed
team_attacking_stats['PASS_ATTEMPTED'] = team_pass_attempted
team_attacking_stats['TAKEONS_ATTEMPTED'] = team_takeons_attempted
team_attacking_stats['TAKEONS_COMPLETED'] = team_takeons_completed
team_attacking_stats['CROSSES_INTO_PA'] = team_crossesintoPA
team_attacking_stats['FOULS_AGAINST'] = team_fouls_against

In [44]:
team_attacking_stats.reset_index(inplace=True)
team_attacking_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [ ]:
upsert_to_snowflake("TEAM_ATTACKING_STATS", team_attacking_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

In [45]:
team_defending_stats = understat_table.copy()

In [46]:
team_defending_stats = team_defending_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [47]:
team_defending_stats = team_defending_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [48]:
team_defending_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [49]:
team_defending_stats['GOALS_CONCEDED'] = team_goals_conceded
team_defending_stats['XG_AGAINST'] = understat_table['XG_AGAINST']
team_defending_stats['NPXG_AGAINST'] = understat_table['NPXG_AGAINST']
team_defending_stats['TACKLES'] = team_tackles
team_defending_stats['TACKLES_WON'] = team_tackles_won
team_defending_stats['FOULS_MADE'] = team_fouls_made
team_defending_stats['INTERCEPTIONS'] = team_interceptions
team_defending_stats['BLOCKED_SHOTS'] = team_blocks_shots
team_defending_stats['BLOCKED_PASSES'] = team_blocks_pass
team_defending_stats['CLEARANCES'] = team_clearances
team_defending_stats['PPDA'] = understat_table['PPDA']

In [50]:
team_defending_stats.reset_index(inplace=True)
team_defending_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [ ]:
upsert_to_snowflake("TEAM_DEFENDING_STATS", team_defending_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

In [51]:
team_misc_stats = understat_table.copy()

In [52]:
team_misc_stats = team_misc_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [53]:
team_misc_stats = team_misc_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [54]:
team_misc_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [55]:
team_misc_stats['AERIALS_WON'] = team_aerials_won
team_misc_stats['AERIALS_LOST'] = team_aerials_lost

In [56]:
team_misc_stats.reset_index(inplace=True)
team_misc_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [ ]:
upsert_to_snowflake("TEAM_MISC_STATS", team_misc_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

In [57]:
df_competitions = fbref_schedule.reset_index()[['COMPETITION', 'SEASON']].drop_duplicates().copy()

In [58]:
df_competitions['COMPETITION_NAME'] = ''
df_competitions['COMPETITION_ACRONYM'] = ''

In [59]:
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_NAME'] = 'English Premier League'
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_ACRONYM'] = 'EPL'

In [ ]:
upsert_to_snowflake("COMPETITIONS", df_competitions, ['COMPETITION','SEASON'])

In [60]:
df_seasons = df_competitions[['SEASON']].drop_duplicates().copy()

In [61]:
df_seasons['SEASON_NAME'] = ''
df_seasons['START_YEAR'] = 0
df_seasons['END_YEAR'] = 0

In [62]:
df_seasons.loc[df_seasons['SEASON'] == 2122, 'SEASON_NAME'] = '2021-2022'
df_seasons.loc[df_seasons['SEASON'] == 2122, 'START_YEAR'] = 2021
df_seasons.loc[df_seasons['SEASON'] == 2122, 'END_YEAR'] = 2022
df_seasons.loc[df_seasons['SEASON'] == 2324, 'SEASON_NAME'] = '2023-2024'
df_seasons.loc[df_seasons['SEASON'] == 2324, 'START_YEAR'] = 2023
df_seasons.loc[df_seasons['SEASON'] == 2324, 'END_YEAR'] = 2024

In [ ]:
upsert_to_snowflake("SEASONS", df_seasons, ['SEASON'])

In [63]:
df_stadiums = fbref_schedule.reset_index()[['home_team', 'SEASON',
            'venue']].drop_duplicates().rename(columns={'home_team': 'TEAM_NAME', 'venue': 'STADIUM'})

In [64]:
df_stadiums = df_stadiums.merge(team_names, on='TEAM_NAME')[['STADIUM','SEASON','TEAM_FBREF_ID']]

In [ ]:
upsert_to_snowflake("STADIUMS", df_stadiums, ['TEAM_FBREF_ID','STADIUM', 'SEASON'])

In [38]:
df_matches = fbref_schedule.reset_index()[['game_id', 'date', 'time', 'home_team', 'away_team', 'COMPETITION', 'SEASON', 'venue',
                                           'week', 'day', 'score', 'home_xg', 'away_xg', 'attendance', 'referee']]

In [39]:
df_matches = df_matches.merge(team_names.rename(columns={'TEAM_FBREF_ID': 'home_team_id', 
                                                         'TEAM_NAME': 'home_team'}), on='home_team', how='left')

In [40]:
df_matches = df_matches.merge(team_names.rename(columns={'TEAM_FBREF_ID': 'away_team_id', 
                                                         'TEAM_NAME': 'away_team'}), on='away_team', how='left')

In [51]:
df_matches = df_matches[df_matches.game_id.notna()]

,game_id,date,time,home_team,away_team,COMPETITION,SEASON,venue,week,day,score,home_xg,away_xg,attendance,referee,home_team_id,away_team_id
0,3a6836b4,2023-08-11,20:00,Burnley,Manchester City,ENG-Premier League,2324,Turf Moor,1,Fri,0–3,0.3,1.9,21572,Craig Pawson,943e8050,b8fd03ef
1,26a7f90c,2023-08-12,12:30,Arsenal,Nott'ham Forest,ENG-Premier League,2324,Emirates Stadium,1,Sat,2–1,0.8,1.2,59984,Michael Oliver,18bb7c10,e4a775cb
2,d6bbf293,2023-08-12,15:00,Bournemouth,West Ham,ENG-Premier League,2324,Vitality Stadium,1,Sat,1–1,1.3,1.1,11245,Peter Bankes,4ba7cbea,7c21e445
3,56a137f7,2023-08-12,15:00,Brighton,Luton Town,ENG-Premier League,2324,The American Express Community Stadium,1,Sat,4–1,4.0,1.5,31872,David Coote,d07537b9,e297cd13
4,15addfc7,2023-08-12,15:00,Everton,Fulham,ENG-Premier League,2324,Goodison Park,1,Sat,0–1,2.7,1.5,39940,Stuart Attwell,d3fd31cc,fd962109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,46bfb645,2023-12-23,15:00,Luton Town,Newcastle Utd,ENG-Premier League,2324,Kenilworth Road Stadium,18,Sat,1–0,1.4,1.4,11042,Darren England,e297cd13,b2b47a98
176,c5548935,2023-12-23,15:00,Nott'ham Forest,Bournemouth,ENG-Premier League,2324,The City Ground,18,Sat,2–3,1.0,1.6,29188,Robert Jones,e4a775cb,4ba7cbea
177,849071fe,2023-12-23,15:00,Tottenham,Everton,ENG-Premier League,2324,Tottenham Hotspur Stadium,18,Sat,2–1,1.5,1.8,61808,Stuart Attwell,361ca564,d3fd31cc
178,328777ed,2023-12-23,12:30,West Ham,Manchester Utd,ENG-Premier League,2324,London Stadium,18,Sat,2–0,1.2,1.0,64472,Simon Hooper,7c21e445,19538871


In [23]:
# Combine df_matches date and time to a dt.datatime configuration
df_matches['date'] = df_matches['date'].astype(str)
df_matches['time'] = df_matches['time'].astype(str)
df_matches['date_time'] = df_matches['date'] + ' ' + df_matches['time']
df_matches['date_time'] = pd.to_datetime(df_matches['date_time'])

In [24]:
# Split score in df_matches by "-" to home_team_score and away_team_score 
df_matches[['home_team_score', 'away_team_score']] = df_matches['score'].str.split('–', expand=True)

In [25]:
df_matches.rename(columns={'game_id': 'MATCH_ID', 'date_time': 'DATE_TIME', 'home_team_id': 'HOME_TEAM_ID', 'away_team_id': 'AWAY_TEAM_ID',
                           'venue': 'STADIUM', 'week': 'GAMEWEEK', 'day': 'DAY','home_team_score': 'HOME_TEAM_SCORE', 
                           'away_team_score':'AWAY_TEAM_SCORE', 'home_xg':'HOME_TEAM_XG','away_xg':'AWAY_TEAM_XG',
                           'attendance':'ATTENDANCE', 'referee': 'REFEREE'}, inplace=True)

In [26]:
df_matches.drop(columns=['date', 'time', 'home_team', 'away_team', 'score'], inplace=True)

In [27]:
df_matches['DATE_TIME'] = df_matches['DATE_TIME'].astype('string')

In [28]:
df_matches

,MATCH_ID,COMPETITION,SEASON,STADIUM,GAMEWEEK,DAY,HOME_TEAM_XG,AWAY_TEAM_XG,ATTENDANCE,REFEREE,HOME_TEAM_ID,AWAY_TEAM_ID,DATE_TIME,HOME_TEAM_SCORE,AWAY_TEAM_SCORE
0,3a6836b4,ENG-Premier League,2324,Turf Moor,1,Fri,0.3,1.9,21572,Craig Pawson,943e8050,b8fd03ef,2023-08-11 20:00:00,0,3
1,26a7f90c,ENG-Premier League,2324,Emirates Stadium,1,Sat,0.8,1.2,59984,Michael Oliver,18bb7c10,e4a775cb,2023-08-12 12:30:00,2,1
2,d6bbf293,ENG-Premier League,2324,Vitality Stadium,1,Sat,1.3,1.1,11245,Peter Bankes,4ba7cbea,7c21e445,2023-08-12 15:00:00,1,1
3,56a137f7,ENG-Premier League,2324,The American Express Community Stadium,1,Sat,4.0,1.5,31872,David Coote,d07537b9,e297cd13,2023-08-12 15:00:00,4,1
4,15addfc7,ENG-Premier League,2324,Goodison Park,1,Sat,2.7,1.5,39940,Stuart Attwell,d3fd31cc,fd962109,2023-08-12 15:00:00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,5db0f25b,ENG-Premier League,2324,St James' Park,17,Sat,3.5,0.3,52055,Samuel Barrott,b2b47a98,fd962109,2023-12-16 15:00:00,3,0
166,e3a46a4e,ENG-Premier League,2324,Emirates Stadium,17,Sun,2.3,0.7,60257,Tim Robinson,18bb7c10,d07537b9,2023-12-17 14:00:00,2,0
167,4a3af0ab,ENG-Premier League,2324,Gtech Community Stadium,17,Sun,0.5,1.4,17034,David Coote,cd051869,8602292d,2023-12-17 14:00:00,1,2
168,18e98d77,ENG-Premier League,2324,Anfield,17,Sun,2.3,0.8,57158,Michael Oliver,822bd0ba,19538871,2023-12-17 16:30:00,0,0


In [86]:
upsert_to_snowflake("MATCHES", df_matches, ['MATCH_ID'])

[12/21/23 17:54:33] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=601783;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=358114;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=861125;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=147806;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=286301;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=40808;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=618687;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=868852;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=287042;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=607126;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[12/21/23 17:54:36] INFO     query execution done                                                     ]8;id=72086;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=888815;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS MATCHES_TEMP]                               ]8;id=954745;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=459785;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=496954;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=63817;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=354313;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=517213;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 550 rows processed.
